In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

import warnings
warnings.filterwarnings('ignore')

# Регрессия

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
О каждой из них мы знаем:
- сколько детей она родила (признак ceb)
- возраст (age)
- длительность получения образования (educ)
- религиозная принадлежность (religion)
- идеальное, по её мнению, количество детей в семье (idlnchld)
- была ли она когда-нибудь замужем (evermarr)
- возраст первого замужества (agefm)
- длительность получения образования мужем (heduc)
- знает ли она о методах контрацепции (knowmeth)
- использует ли она методы контрацепции (usemeth)
- живёт ли она в городе (urban)
- есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

In [2]:
df = pd.read_csv('botswana.tsv', sep = '\t')
df.head(10)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
5,1,30,5,spirit,5.0,1.0,0.0,1,24.0,7.0,1,1.0,0.0,0.0,0.0
6,3,42,4,other,3.0,1.0,0.0,1,15.0,11.0,1,1.0,0.0,1.0,0.0
7,1,36,7,other,4.0,1.0,1.0,1,24.0,9.0,1,1.0,0.0,0.0,0.0
8,4,37,16,catholic,4.0,1.0,1.0,1,26.0,17.0,1,1.0,1.0,1.0,1.0
9,1,34,5,protestant,4.0,1.0,1.0,1,18.0,3.0,1,0.0,1.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
ceb         4361 non-null int64
age         4361 non-null int64
educ        4361 non-null int64
religion    4361 non-null object
idlnchld    4241 non-null float64
knowmeth    4354 non-null float64
usemeth     4290 non-null float64
evermarr    4361 non-null int64
agefm       2079 non-null float64
heduc       1956 non-null float64
urban       4361 non-null int64
electric    4358 non-null float64
radio       4359 non-null float64
tv          4359 non-null float64
bicycle     4358 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.1+ KB


## Количество религий
Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [4]:
rels = df['religion'].unique()
print(rels)

['catholic' 'protestant' 'spirit' 'other']


In [5]:
len(rels)

4

## Пропущенные значения
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [6]:
df_drop = df.dropna()
df_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1834 entries, 1 to 4360
Data columns (total 15 columns):
ceb         1834 non-null int64
age         1834 non-null int64
educ        1834 non-null int64
religion    1834 non-null object
idlnchld    1834 non-null float64
knowmeth    1834 non-null float64
usemeth     1834 non-null float64
evermarr    1834 non-null int64
agefm       1834 non-null float64
heduc       1834 non-null float64
urban       1834 non-null int64
electric    1834 non-null float64
radio       1834 non-null float64
tv          1834 non-null float64
bicycle     1834 non-null float64
dtypes: float64(9), int64(5), object(1)
memory usage: 229.2+ KB


## Обработка пропусков
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

- создать новый бинарный признак
 $$
x_2 = \left\{\begin{matrix}1, x_1 = не-применимо
\\ 0, иначе
\end{matrix}\right.
 $$
- заменить "не применимо" в $x_1$ на произвольную константу $c$, которая среди других значений $x_1$ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида 
$$y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,$$
на тех объектах, где x1x_1x1​ было измерено, регрессионное уравнение примет вид 
$$y=\beta_0 + \beta_1 x,$$ а там, где $x_1$ было "не применимо", получится 
$$y=\beta_0 + \beta_1 c + \beta_2.$$ 
Выбор $c$ влияет только на значение и интерпретацию $\beta_2$, но не $\beta_1$.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

- Создайте признак nevermarr, равный единице там, где в agefm пропуски.
- Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице $X$ будет мультиколлинеарность.
- Замените NaN в признаке agefm на $c_{agefm}=0$.
- У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [7]:
df['nevermarr'] = 0
df.loc[df.agefm.isna(),'nevermarr'] = 1
df = df.drop('evermarr', axis = 1)
df.agefm.fillna(0, inplace = True)
df.loc[((df.nevermarr == 1)&(df.heduc.isna())),'heduc'] = -1

In [8]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [9]:
df.heduc.isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

## Обработка пропусков. Часть 2
Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}=-1$, $c_{heduc_2}=-2$ (значение -1 мы уже использовали), $c_{usemeth}=-1$.

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [10]:
def create_noans_column(df, old_column, new_column, new_value):
    df[new_column] = 0
    df.loc[df[old_column].isna(),new_column] = 1
    df.loc[:,old_column].fillna(new_value, inplace=True)

In [11]:
create_noans_column(df, 'idlnchld', 'idlnchld_noans', -1)
create_noans_column(df, 'heduc', 'heduc_noans', -2)
create_noans_column(df, 'usemeth', 'usemeth_noans', -1)

In [12]:
df = df.dropna(subset = ['knowmeth', 'electric', 'radio', 'tv', 'bicycle'])

In [13]:
df.shape[0]*df.shape[1]

78264

## Коэффициент детерминации
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$? Округлите до трёх знаков после десятичной точки.

In [14]:
cols = df.columns[1:]
result = smf.ols(formula="ceb ~ " + " + ".join(cols), data=df)
fitted = result.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sat, 07 Mar 2020   Prob (F-statistic):               0.00
Time:                        11:53:06   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [15]:
np.round(fitted.rsquared, 3)

0.644

## Гомоскедастичность
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

In [16]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


## Удаление признаков
Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [17]:
cols_clear = df.columns[1:].tolist()
for name in ['religion', 'radio', 'tv']:
    cols_clear.remove(name)

result_hc1 = smf.ols(formula="ceb ~ " + " + ".join(cols_clear), data=df)
fitted_hc1 = result_hc1.fit(cov_type='HC1')
print("F=%f, p=%.4f, k1=%f" % fitted.compare_f_test(fitted_hc1))

F=0.919236, p=0.4672, k1=5.000000


In [18]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_hc1.resid, fitted_hc1.model.exog)[1])

Breusch-Pagan test: p=0.000000


## Удаление признаков. Часть 2
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1c_{usemeth }=-1cusemeth​=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans. 

In [19]:
cols_clear = df.columns[1:].tolist()
for name in ['religion', 'radio', 'tv', 'usemeth_noans', 'usemeth']:
    cols_clear.remove(name)

result_hc1_V2 = smf.ols(formula="ceb ~ " + " + ".join(cols_clear), data=df)
fitted_hc1_V2 = result_hc1_V2.fit(cov_type='HC1')
fitted_hc1.compare_f_test(fitted_hc1_V2)[1]

3.1552009480371243e-40

## Дов интервалы
Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

In [20]:
print(fitted_hc1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sat, 07 Mar 2020   Prob (F-statistic):               0.00
Time:                        11:53:06   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

## Результат
Был закреплен материал из лекций. Особенно выводы по получившимся коэффициентам и доверительным интервалам. 